# Dataset Construction

In [1]:
import numpy as np
import pandas as pd
from torch_geometric_temporal.signal import StaticGraphTemporalSignal, DynamicGraphTemporalSignal
import torch
from typing import Union
import glob
from natsort import natsorted
import random

random.seed(42)

class SP500CorrelationsDatasetLoader(object):
    def __init__(self, corr_name, corr_scope):
        self._read_csv(corr_name, corr_scope)

    def _read_csv(self, corr_name, corr_scope):
        match corr_scope:
            case 'global':
                self._correlation_matrices = [np.loadtxt(f'{corr_name}/{corr_scope}_corr.csv', delimiter=',')]
            case 'local':
                self._correlation_matrices = []
                corr_files = natsorted(glob.glob(f'{corr_name}/local_corr_*.csv'))
                for corr_file in corr_files:
                    matrix = np.loadtxt(corr_file, delimiter=',')
                    self._correlation_matrices.append(matrix)
        
        df = pd.read_csv('s&p500.csv')
        df = df.set_index('Date')
        data = torch.from_numpy(df.to_numpy()).to(torch.float32)

        # Round data size to nearest multiple of batch_size
        self.days_in_quarter = 64
        num_quarters = data.size(0) // self.days_in_quarter
        num_days = num_quarters * self.days_in_quarter
        data = data[:num_days]
        
        # z-score normalization with training data following GERU
        train_days = int(0.8 * num_quarters) * self.days_in_quarter
        data = (data - data[:train_days].mean(dim=0)) / data[:train_days].std(dim=0)
        data = data.numpy()

        # Add percent change features
        p_chg = data / np.roll(data, 1, axis=0) - 1
        p_chg[0] = 0.0
        p_chg_3 = data / np.roll(data, 3, axis=0) - 1
        p_chg_3[0:3] = 0.0
        p_chg_6 = data / np.roll(data, 6, axis=0) - 1
        p_chg_6[0:6] = 0.0

        data = np.stack([data, p_chg, p_chg_3, p_chg_6], axis=-1)
        print('data.shape', data.shape)

        assert(not np.any(np.isnan(data)))
        self._dataset = data

    def _get_edges(self):
        if len(self._correlation_matrices) == 1:
            self._edges = np.array(np.ones_like(self._correlation_matrices[0]).nonzero())
        else:
            self._edges = []
            for time in range(self._dataset.shape[0] - self.lags):
                corr_index = max(0, time // self.days_in_quarter - 1)
                self._edges.append(
                    np.array(np.ones_like(self._correlation_matrices[corr_index]).nonzero())
                )

    def _get_edge_weights(self):
        if len(self._correlation_matrices) == 1:
            self._edge_weights = self._correlation_matrices[0].flatten()
        else:
            self._edge_weights = []
            for time in range(self._dataset.shape[0] - self.lags):
                corr_index = max(0, time // self.days_in_quarter - 1)
                self._edge_weights.append(
                    np.array(self._correlation_matrices[corr_index]).flatten()
                )

    def _get_targets_and_features(self):
        stacked_target = self._dataset
        # print(stacked_target.shape)
        self.features = [
            stacked_target[i : i + self.lags, :]
            for i in range(stacked_target.shape[0] - self.lags)
        ]
        # predict next-day stock movement
        self.targets = [
            (stacked_target[i + self.lags, :, 0]).T
            for i in range(stacked_target.shape[0] - self.lags)
        ]

    def get_dataset(self, lags) -> Union[StaticGraphTemporalSignal, DynamicGraphTemporalSignal]:
        """Returning the data iterator.
        """
        self.lags = lags
        self._get_edges()
        self._get_edge_weights()
        self._get_targets_and_features()
        dataset = (DynamicGraphTemporalSignal if type(self._edges) == list else StaticGraphTemporalSignal)(
            self._edges, self._edge_weights, self.features, self.targets
        )
        return dataset

In [2]:
from torch_geometric_temporal.signal import temporal_signal_split

# device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device = 'cpu'

corr_name = 'mi'
corr_scope = 'global'
loader = SP500CorrelationsDatasetLoader(corr_name=corr_name, corr_scope=corr_scope)

lags = 6
dataset = loader.get_dataset(lags)

train_dataset, test_val_dataset = temporal_signal_split(dataset, train_ratio=0.8)
val_dataset, test_dataset = temporal_signal_split(test_val_dataset, train_ratio=0.5)

data.shape (2496, 472, 4)


In [3]:
print(train_dataset.features[0].shape)
print(train_dataset.targets[0].shape)

(6, 472, 4)
(472,)


# Differential Graph Transformer

In [4]:
import torch
from torch_geometric.utils import to_dense_adj
from torch_geometric.nn.conv import MessagePassing
import torch.nn as nn
import torch.nn.functional as F


class DGAttn(MessagePassing):
    r"""An implementation of the Diffusion Convolution Layer.
    For details see: `"Diffusion Convolutional Recurrent Neural Network:
    Data-Driven Traffic Forecasting" <https://arxiv.org/abs/1707.01926>`_

    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        K (int): Filter size :math:`K`.
        bias (bool, optional): If set to :obj:`False`, the layer
            will not learn an additive bias (default :obj:`True`).

    """

    def __init__(self, in_channels, out_channels, num_heads=1, diff_attn=True):
        super(DGAttn, self).__init__(aggr="add", flow="source_to_target")
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_heads = num_heads
        self.head_dim = out_channels // self.num_heads
        self.diff_attn = diff_attn
        if self.diff_attn:
            self.Q = nn.Linear(in_channels, out_channels)
            self.K = nn.Linear(in_channels, out_channels)
        self.V = nn.Linear(in_channels, out_channels)
        self.ffn = nn.Linear(out_channels, out_channels)
        self.ln = nn.RMSNorm(self.head_dim, eps=1e-5, elementwise_affine=True)

        if self.diff_attn:
            self.lambda_init = 0.2
            self.lambda_q1 = nn.Parameter(torch.zeros(self.head_dim, dtype=torch.float32).normal_(mean=0,std=0.1))
            self.lambda_k1 = nn.Parameter(torch.zeros(self.head_dim, dtype=torch.float32).normal_(mean=0,std=0.1))
            self.lambda_q2 = nn.Parameter(torch.zeros(self.head_dim, dtype=torch.float32).normal_(mean=0,std=0.1))
            self.lambda_k2 = nn.Parameter(torch.zeros(self.head_dim, dtype=torch.float32).normal_(mean=0,std=0.1))

        # self.__reset_parameters()

    # def __reset_parameters(self):
        # torch.nn.init.xavier_uniform_(self.V)

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def forward(
        self,
        X: torch.FloatTensor,
        edge_index: torch.LongTensor,
        edge_weight: torch.FloatTensor,
    ) -> torch.FloatTensor:
        r"""Making a forward pass. If edge weights are not present the forward pass
        defaults to an unweighted graph.

        Arg types:
            * **X** (PyTorch Float Tensor) - Node features.
            * **edge_index** (PyTorch Long Tensor) - Graph edge indices.
            * **edge_weight** (PyTorch Long Tensor, optional) - Edge weight vector.

        Return types:
            * **H** (PyTorch Float Tensor) - Hidden state matrix for all nodes.
        """
        A = to_dense_adj(edge_index, edge_attr=edge_weight).squeeze()
        a1 = F.softmax(A, dim=-1)
        if self.diff_attn:
            q = self.Q(X)
            k = self.K(X)
            a2 = F.softmax(torch.matmul(q, k.transpose(-1, -2)) * (self.head_dim ** -0.5), dim=-1)
            lambda_1 = torch.exp(torch.sum(self.lambda_q1 * self.lambda_k1, dim=-1).float()).type_as(q)
            lambda_2 = torch.exp(torch.sum(self.lambda_q2 * self.lambda_k2, dim=-1).float()).type_as(q)
            lambda_full = lambda_1 - lambda_2 + self.lambda_init
            a = a1 - lambda_full * a2
        else:
            a = a1
        v = self.V(X)
        attn = torch.matmul(a, v)
        attn = self.ln(attn)
        if self.diff_attn:
            attn = attn * (1 - self.lambda_init)
        attn = self.ffn(attn)
        return attn


class DGRNN(torch.nn.Module):
    r"""An implementation of the Diffusion Convolutional Gated Recurrent Unit.
    For details see: `"Diffusion Convolutional Recurrent Neural Network:
    Data-Driven Traffic Forecasting" <https://arxiv.org/abs/1707.01926>`_

    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        num_heads (int): Number of attention heads.
    """

    def __init__(self, in_channels: int, out_channels: int, num_heads: int, diff_attn: bool):
        super(DGRNN, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_heads = num_heads
        self.diff_attn = diff_attn,

        self._create_parameters_and_layers()

    def _create_update_gate_parameters_and_layers(self):
        self.conv_x_z = DGAttn(
            in_channels=self.in_channels + self.out_channels,
            out_channels=self.out_channels,
            num_heads=self.num_heads,
            diff_attn=self.diff_attn,
        )

    def _create_reset_gate_parameters_and_layers(self):
        self.conv_x_r = DGAttn(
            in_channels=self.in_channels + self.out_channels,
            out_channels=self.out_channels,
            num_heads=self.num_heads,
            diff_attn=self.diff_attn,
        )

    def _create_candidate_state_parameters_and_layers(self):
        self.conv_x_h = DGAttn(
            in_channels=self.in_channels + self.out_channels,
            out_channels=self.out_channels,
            num_heads=self.num_heads,
            diff_attn=self.diff_attn,
        )

    def _create_parameters_and_layers(self):
        self._create_update_gate_parameters_and_layers()
        self._create_reset_gate_parameters_and_layers()
        self._create_candidate_state_parameters_and_layers()

    def _set_hidden_state(self, X, H):
        if H is None:
            H = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return H

    def _calculate_update_gate(self, X, edge_index, edge_weight, H):
        Z = torch.cat([X, H], dim=1)
        Z = self.conv_x_z(Z, edge_index, edge_weight)
        Z = torch.sigmoid(Z)
        return Z

    def _calculate_reset_gate(self, X, edge_index, edge_weight, H):
        R = torch.cat([X, H], dim=1)
        R = self.conv_x_r(R, edge_index, edge_weight)
        R = torch.sigmoid(R)
        return R

    def _calculate_candidate_state(self, X, edge_index, edge_weight, H, R):
        H_tilde = torch.cat([X, H * R], dim=1)
        H_tilde = self.conv_x_h(H_tilde, edge_index, edge_weight)
        H_tilde = torch.tanh(H_tilde)
        return H_tilde

    def _calculate_hidden_state(self, Z, H, H_tilde):
        H = Z * H + (1 - Z) * H_tilde
        return H

    def forward(
        self,
        X: torch.FloatTensor,
        edge_index: torch.LongTensor,
        edge_weight: torch.FloatTensor = None,
        H: torch.FloatTensor = None,
    ) -> torch.FloatTensor:
        r"""Making a forward pass. If edge weights are not present the forward pass
        defaults to an unweighted graph. If the hidden state matrix is not present
        when the forward pass is called it is initialized with zeros.

        Arg types:
            * **X** (PyTorch Float Tensor) - Node features.
            * **edge_index** (PyTorch Long Tensor) - Graph edge indices.
            * **edge_weight** (PyTorch Long Tensor, optional) - Edge weight vector.
            * **H** (PyTorch Float Tensor, optional) - Hidden state matrix for all nodes.

        Return types:
            * **H** (PyTorch Float Tensor) - Hidden state matrix for all nodes.
        """
        H = self._set_hidden_state(X, H)
        # print('X.shape', X.shape)
        # print('H.shape', H.shape)
        Z = self._calculate_update_gate(X, edge_index, edge_weight, H)
        R = self._calculate_reset_gate(X, edge_index, edge_weight, H)
        H_tilde = self._calculate_candidate_state(X, edge_index, edge_weight, H, R)
        H = self._calculate_hidden_state(Z, H, H_tilde)
        return H


# Plain RNN

In [5]:
class GRU(torch.nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super(GRU, self).__init__()
        self.rnn = nn.GRU(input_size=in_channels, hidden_size=out_channels, num_layers=2, batch_first=True)

    def forward(
        self,
        X: torch.FloatTensor,
        edge_index: torch.LongTensor,
        edge_weight: torch.FloatTensor = None,
        H: torch.FloatTensor = None,
    ) -> torch.FloatTensor:
        outputs, H = self.rnn(X, H)
        return H[-1]

# RGCN

In [6]:
import torch
import torch.nn.functional as F

class RecurrentGNN(torch.nn.Module):
    def __init__(self, gnn, node_features, hidden_size=32, **kwargs):
        super(RecurrentGNN, self).__init__()
        if gnn.__name__ == "DCRNN":
            self.recurrent = gnn(in_channels=node_features, out_channels=hidden_size, **kwargs)
        elif gnn.__name__ == "DGRNN":
            self.recurrent = gnn(in_channels=node_features, out_channels=hidden_size, **kwargs)
        elif gnn.__name__ == "GRU":
            self.recurrent = gnn(in_channels=node_features, out_channels=hidden_size, **kwargs)
        else:
            raise ValueError(f"Invalid GNN type {gnn.__name__}")
        self.linear = torch.nn.Linear(hidden_size, 1)

    def forward(self, x, edge_index, edge_weight, hidden):
        h0 = self.recurrent(x, edge_index, edge_weight, hidden)
        h = F.relu(h0)
        h = self.linear(h)
        return (h, h0)

In [7]:
import math

def rmse(y_hat, y):
    return math.sqrt(F.mse_loss(y_hat, y).item())

def mae(y_hat, y):
    return F.l1_loss(y_hat, y).item()

In [8]:
import wandb

def eval(epoch, model, eval_dataset, eval_name):
    model.eval()
    with torch.no_grad():
        y_hats = []
        ys = []
        for snapshot in eval_dataset:
            X = snapshot.x.to(device)
            h = None
            if model.recurrent.__class__.__name__ == "GRU":
                y_hat, _ = model(X.transpose(0, 1), snapshot.edge_index.to(device), snapshot.edge_attr.to(device), h)
            else:
                for x in X:
                    y_hat, h = model(x, snapshot.edge_index.to(device), snapshot.edge_attr.to(device), h)
            y_hats.append(y_hat)
            ys.append(snapshot.y.to(device))
        y_hats, ys = torch.stack(list(y_hats)).squeeze().to(device), torch.stack(list(ys)).to(device)
        eval_rmse = rmse(y_hats, ys)
        eval_mae = mae(y_hats, ys)
        wandb.log({"epoch": epoch,
                f"{eval_name}/rmse": eval_rmse,
                f"{eval_name}/mae": eval_mae })
        print(f'Epoch {epoch}, {eval_name}/rmse: {eval_rmse}, {eval_name}/mae: {eval_mae}')
        return (eval_rmse, eval_mae)

In [9]:
from tqdm import tqdm
import wandb
from torch_geometric_temporal.nn.recurrent import DCRNN

gnn = GRU

model = RecurrentGNN(gnn = gnn, node_features = 4).to(device)

if gnn.__name__ == 'GRU':
    model_name = f'{gnn.__name__}'
elif gnn.__name__ == 'DCRNN':
    model_name = f'{gnn.__name__}_{corr_name}_{corr_scope}'
elif gnn.__name__ == 'DGRNN':
    model_name = f'{gnn.__name__}_{corr_name}_{corr_scope}{"_diff_attn" if model.recurrent.diff_attn else ""}'

lr = 1e-3
num_epochs = 50
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
track_with_wandb = True

if track_with_wandb:
    wandb.init(project="cs224w-stock-market-prediction", config={
        "dataset": "S&P500",
        "corr_name": corr_name,
        "corr_scope": corr_scope,
        "learning_rate": lr,
        "epochs": num_epochs,
        "architecture": gnn.__name__,
    })

best_val_rmse = float('inf')

batch_size = 64

def split_into_batches(lst, batch_size):
    return [lst[i:i + batch_size] for i in range(0, len(lst), batch_size)]

train_samples = list(train_dataset)
train_batches = split_into_batches(train_samples, batch_size)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    random.shuffle(train_batches)
    for step, batch in tqdm(enumerate(train_batches), total=len(train_batches), desc=f'Epoch {epoch}'):
        y_hats = []
        ys = []
        for snapshot in batch:
            X = snapshot.x.to(device)
            h = None
            if model.recurrent.__class__.__name__ == "GRU":
                y_hat, _ = model(X.transpose(0, 1), snapshot.edge_index.to(device), snapshot.edge_attr.to(device), h)
            else:
                for x in X:
                    y_hat, h = model(x, snapshot.edge_index.to(device), snapshot.edge_attr.to(device), h)
            y_hats.append(y_hat)
            ys.append(snapshot.y.to(device))
        y_hats, ys = torch.stack(y_hats).squeeze(), torch.stack(ys)
        loss = F.mse_loss(y_hats, ys)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        wandb.log({"epoch": epoch,
                   "step": epoch * len(train_samples) + step,
                    "train/loss": loss.item() })
    train_loss /= len(train_samples)

    if track_with_wandb:
        val_rmse, val_mae = eval(epoch, model, val_dataset, 'val')
        if val_rmse < best_val_rmse:
            best_rmse = val_rmse
            torch.save(model.state_dict(), f'{model_name}.pth')


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kevinxli. Use `wandb login --relogin` to force relogin


Epoch 0: 100%|██████████| 32/32 [00:15<00:00,  2.09it/s]


Epoch 0, val/rmse: 1.5263910078165706, val/mae: 1.0989220142364502


Epoch 1: 100%|██████████| 32/32 [00:15<00:00,  2.04it/s]


Epoch 1, val/rmse: 1.1081476541216275, val/mae: 0.5569794774055481


...

Epoch 47, val/rmse: 0.7225406863626448, val/mae: 0.14256401360034943


Epoch 48: 100%|██████████| 32/32 [00:19<00:00,  1.67it/s]


Epoch 48, val/rmse: 0.720767093290359, val/mae: 0.14812614023685455


Epoch 49: 100%|██████████| 32/32 [00:20<00:00,  1.60it/s]


Epoch 49, val/rmse: 0.7201615860208117, val/mae: 0.14087459444999695


In [10]:
if track_with_wandb:
    best_model = RecurrentGNN(gnn = gnn, node_features = 4).to(device)
    best_model.load_state_dict(torch.load(f'{model_name}.pth', weights_only=True))
    test_rmse, test_mae = eval(epoch, best_model, test_dataset, 'test')


Epoch 49, test/rmse: 3.268968686434329, test/mae: 0.5551874041557312
